In [1]:
# Install redis library
!pip install redis


In [2]:
# Import necessary libraries
import redis
import json  # For working with JSON data

In [3]:
# Connect to Redis Cloud
def connect_to_redis():
    client = redis.StrictRedis(
        host="xxx.redis-cloud.com",  # Your Redis host
        port=xxxx,  # Your Redis port
        username="replace with your username",  # Your Redis username
        password="replace with your password",  # Your Redis password
        decode_responses=True  # Ensures responses are returned as strings
    )
    return client

# Test the connection
client = connect_to_redis()

try:
    client.ping()  # Send a ping to check the connection
    print("Connected to Redis successfully!")
except redis.exceptions.ConnectionError as e:
    print(f"Failed to connect to Redis: {e}")


Connected to Redis successfully!


In [6]:
# Load JSON data from the uploaded file
file_path = "metro.json"
with open(file_path, "r") as file:
    metro_data = json.load(file)

# Extract station nodes
stations = metro_data["nodes"]
print(f"Loaded {len(stations)} stations.")

Loaded 314 stations.


In [9]:
import requests
# Function to fetch weather data using Open-Meteo
def fetch_weather_data(latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json().get("current_weather")
    else:
        print(f"Failed to fetch weather data for coordinates ({latitude}, {longitude}): {response.status_code}")
        return None


In [10]:
# Cache weather data in Redis
def cache_weather_data(station_id, weather_data):
    key = f"weather:{station_id}"
    client.set(key, json.dumps(weather_data), ex=3600)  # Cache for 1 hour
    print(f"Weather data cached for station ID {station_id}.")


In [11]:
# Fetch and cache weather data for all stations
for station in stations:
    latitude = station["latitude"]
    longitude = station["longitude"]
    station_id = station["id"]
    
    # Fetch weather data
    weather = fetch_weather_data(latitude, longitude)
    
    # Cache the data if it was fetched successfully
    if weather:
        cache_weather_data(station_id, weather)

Weather data cached for station ID 12388528.
Weather data cached for station ID 12388534.
Weather data cached for station ID 27332092.
Weather data cached for station ID 27362238.
Weather data cached for station ID 27362240.
Weather data cached for station ID 27362249.
Weather data cached for station ID 27362251.
Weather data cached for station ID 27362257.
Weather data cached for station ID 27362258.
Weather data cached for station ID 27362259.
Weather data cached for station ID 27362260.
Weather data cached for station ID 27371862.
Weather data cached for station ID 27371867.
Weather data cached for station ID 27371876.
Weather data cached for station ID 27371889.
Weather data cached for station ID 27371892.
Weather data cached for station ID 27371900.
Weather data cached for station ID 29588070.
Weather data cached for station ID 34909170.
Weather data cached for station ID 34909171.
Weather data cached for station ID 34909187.
Weather data cached for station ID 134299001.
Weather d

In [12]:
# Retrieve cached weather data
def get_cached_weather_data(station_id):
    key = f"weather:{station_id}"
    data = client.get(key)
    return json.loads(data) if data else None

# Example: Retrieve weather data for a specific station
station_id = 12388528  # Replace with a station ID from your JSON
cached_weather = get_cached_weather_data(station_id)
if cached_weather:
    print(f"Cached weather data for station ID {station_id}:")
    print(cached_weather)
else:
    print(f"No cached weather data for station ID {station_id}.")

Cached weather data for station ID 12388528:
{'time': '2025-01-11T19:30', 'interval': 900, 'temperature': 0.9, 'windspeed': 6.9, 'winddirection': 332, 'is_day': 0, 'weathercode': 3}
